In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 23.4 MB/s eta 0:00:00


# Section 1: Load and Preprocess Data

In [7]:
import torch
from torch.utils.data import TensorDataset, DataLoader
from transformers import BartTokenizer
from sklearn.model_selection import train_test_split
import pandas as pd

### Load data

In [8]:
df = pd.read_csv('/content/extract_data.csv')
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')

### Tokenization functions

In [9]:
def tokenize_text(text):
    return tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=512, truncation=True, padding='max_length', return_attention_mask=True)

def tokenize_summary(text):
    return tokenizer.encode(text, return_tensors="pt", max_length=280, truncation=True, padding='max_length', return_attention_mask=True)

### Apply tokenization

In [10]:
df['TokenizedText'] = df['articles'].apply(tokenize_text)
df['TokenizedSummary'] = df['summaries'].apply(tokenize_summary)


In [11]:
# Split data
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [17]:
# Convert tokenized data to PyTorch tensors
X_train = torch.stack([seq.squeeze() for seq in train_df['TokenizedText']])
Y_train = torch.stack([seq.squeeze() for seq in train_df['TokenizedSummary']])
X_test = torch.stack([seq.squeeze() for seq in test_df['TokenizedText']])
Y_test = torch.stack([seq.squeeze() for seq in test_df['TokenizedSummary']])

In [18]:
# Define a DataLoader for batching data
train_dataset = TensorDataset(X_train, Y_train)
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_dataset = TensorDataset(X_test, Y_test)
test_dataloader = DataLoader(test_dataset, batch_size=4)



# Section 2: Initialize BART Model

In [19]:
from transformers import BartForConditionalGeneration

# Initialize BART model
model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=

# Section 3: Define Optimizer and Scheduler

In [20]:
from transformers import AdamW, get_linear_schedule_with_warmup

# Define optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=50, num_training_steps=len(train_dataloader) * 10)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


# Section 4: Training Loop

In [21]:
from torch.cuda.amp import autocast, GradScaler
from torch.nn.utils import clip_grad_norm_
from tqdm import tqdm

# Create GradScaler for mixed-precision training
scaler = GradScaler()

# Training loop
for epoch in range(2):
    model.train()
    total_loss = 0.0
    optimizer.zero_grad()

    for step, batch in enumerate(tqdm(train_dataloader, desc="Training")):
        inputs, targets = batch[0].to(device), batch[1].to(device)

        with autocast():
            outputs = model(input_ids=inputs, decoder_input_ids=targets, labels=targets)
            loss = outputs.loss / 20  # gradient accumulation

        scaler.scale(loss).backward()
        if (step + 1) % 20 == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()

        total_loss += loss.item()

    print(f"Epoch {epoch + 1}/{2}, Train Loss: {total_loss / len(train_dataloader):.4f}")

Training: 100%|██████████| 445/445 [01:20<00:00,  5.51it/s]


Epoch 1/2, Train Loss: 0.5613


Training: 100%|██████████| 445/445 [01:20<00:00,  5.54it/s]

Epoch 2/2, Train Loss: 0.5587


In [23]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=b2ba9ef10955b3178d9747b2e5aa859440f93e0636ecf60e14857c0bf0597759
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


# Section 5: Evaluation

In [26]:
from rouge_score import rouge_scorer

In [27]:
model.eval()
rouge_scorer = rouge_scorer.RougeScorer(['rouge1'])
rouge1_precision_scores = []

for batch in tqdm(test_dataloader, desc="Evaluating Test"):
    inputs, targets = batch[0].to(device), batch[1].to(device)
    outputs = model.generate(input_ids=inputs, max_length=150, num_beams=17, length_penalty=2.0, early_stopping=False)

    for output, target in zip(outputs, targets):
        output_text = tokenizer.decode(output, skip_special_tokens=True)
        target_text = tokenizer.decode(target, skip_special_tokens=True)
        rouge_scores = rouge_scorer.score(output_text, target_text)
        rouge1_precision_scores.append(rouge_scores['rouge1'].precision)

Evaluating Test: 100%|██████████| 112/112 [11:10<00:00,  5.98s/it]


In [29]:
import pandas as pd
from tqdm import tqdm
from rouge_score import rouge_scorer

# Assuming you have already defined test_dataloader, model, and tokenizer

# Set the model to evaluation mode
model.eval()

# Initialize Rouge scorer
rouge_scorer = rouge_scorer.RougeScorer(['rouge1'])

# Initialize lists to store results
test_articles = []
actual_summaries = []
predicted_summaries = []
rouge1_precision_scores = []

# Loop through the test data
for batch in tqdm(test_dataloader, desc="Evaluating Test"):
    inputs, targets = batch[0].to(device), batch[1].to(device)
    outputs = model.generate(input_ids=inputs, max_length=150, num_beams=17, length_penalty=2.0, early_stopping=False)

    for output, target, input_text in zip(outputs, targets, inputs):
        # Calculate ROUGE-1 precision for each sample
        output_text = tokenizer.decode(output, skip_special_tokens=True)
        target_text = tokenizer.decode(target, skip_special_tokens=True)
        rouge_scores = rouge_scorer.score(output_text, target_text)
        rouge1_precision_scores.append(rouge_scores['rouge1'].precision)

        # Append tokenized text, actual summaries, and predicted summaries
        test_articles.append(tokenizer.decode(input_text, skip_special_tokens=True))
        actual_summaries.append(target_text)
        predicted_summaries.append(output_text)

# Create a dictionary with the extracted data
data = {
    'Article': test_articles,
    'Actual Summary': actual_summaries,
    'Predicted Summary': predicted_summaries,
    'ROUGE-1 Precision': rouge1_precision_scores,
}

# Create a Pandas DataFrame from the dictionary
results_df = pd.DataFrame(data)

# Display the DataFrame
pd.set_option('display.max_colwidth', None)
print(results_df.head(5))


Evaluating Test: 100%|██████████| 112/112 [11:29<00:00,  6.16s/it]

# Section 6: Results DataFrame

In [31]:
import pandas as pd

results_df = pd.DataFrame({
    'ROUGE-1 Precision': rouge1_precision_scores
})

# Display the DataFrame
pd.set_option('display.max_colwidth', None)
results_df.head(5)


,ROUGE-1 Precision
0,0.433498
1,0.485507
2,0.506757
3,0.515385
4,0.699248


In [30]:
results_df.head(5)

,Article,Actual Summary,Predicted Summary,ROUGE-1 Precision
0,"summarize: Tsunami slows Sri Lanka's growth\n\nSri Lanka's president has launched a reconstruction drive worth $3.5bn (Â£1.8bn) by appealing for peace and national unity.\n\nPresident Kumaratunga said it was now important to find a peaceful solution to years of internal conflict. Meanwhile, the International Monetary Fund (IMF) said damage from the tsunami would cut one percentage point from Sri Lanka's economic growth this year. It estimated the wave left physical damage equal to 6.5% of the economy.\n\nSeparately, the International Labour Organisation (ILO) said that at least one million people have lost their livelihoods in Sri Lanka and Indonesia alone. It called for action to create jobs. President Kumaratunga attended a ceremony in the southern town of Hambantota. She was joined by government and opposition politicians, together with Buddhist, Hindu, Muslim and Christian clergy.\n\nPrime Minister Mahinda Rajapakse laid the foundation stone on a new housing project intended to provide 6,000 homes for survivors of the tsunami. Mrs Kumaratunga called for the tragedy to be ""the start of a new beginning to rebuild our nation"". ""We are a country blessed with so many natural resources and we have not made use of them fully. Instead we have been squabbling, fighting,"" she added. Norway's peace negotiator Erik Solheim is due to arrive on Wednesday to try to revive peace talks in the decades-long conflict between government forces and the Tamil Tigers, who want a separate state in the north east of the country. Reconstruction efforts in eastern Sri Lanka have been hampered by tensions between the two sides.\n\nThe IMF said that the Sri Lankan authorities' initial estimates have put the physical damage at $1.3 to $1.5bn, but added that the implications for the economy were much wider than this. ""The broader macroeconomic impact will clearly be substantial but the details are difficult to assess at this early stage,"" the IMF said. Growth, inflation, the balance of payments and foreign exchange reserves are all expected to show the effects of lost businesses and reconstruction costs. ""The fishing industry has been devastated, agricultural production may be affected and tourism will suffer, especially in the short term,"" the report said. The ILO estimated that 400,000 Sri Lankans have lost their jobs, mostly in these three industries. Earnings from tourism this year are expected to be 15% lower than last year. Economic growth this","Meanwhile, the International Monetary Fund (IMF) said damage from the tsunami would cut one percentage point from Sri Lanka's economic growth this year.The IMF said that the Sri Lankan authorities' initial estimates have put the physical damage at $1.3 to $1.5bn, but added that the implications for the economy were much wider than this.President Kumaratunga said it was now important to find a peaceful solution to years of internal conflict.Sri Lanka's president has launched a reconstruction drive worth $3.5bn (Â£1.8bn) by appealing for peace and national unity.Separately, the International Labour Organisation (ILO) said that at least one million people have lost their livelihoods in Sri Lanka and Indonesia alone.The ILO estimated that 400,000 Sri Lankans have lost their jobs, mostly in these three industries.Reconstruction efforts in eastern Sri Lanka have been hampered by tensions between the two sides.Last week, the IMF approved Sri Lanka's request for a freeze on loan repayments.Growth, inflation, the balance of payments and foreign exchange reserves are all expected to show the effects of lost businesses and reconstruction costs.Earnings from tourism this year are expected to be 15% lower than last year.","summarize: Tsunami slows Sri Lanka's growthSri Lanka's president has launched a reconstruction drive worth $3.5bn (Â£1.8bn) by appealing for peace and national unity. The International Monetary Fund (IMF) said damage from the tsun